In [1]:
# 调不通 不知道为啥

# https://www.cnblogs.com/upright/p/6136265.html
# https://blog.csdn.net/weixin_41787032/article/details/79775469
# 先修改G:\Anaconda3\My_Jupyter\nets下的Alexnet
# 建立在21 22的基础上

In [2]:
# 方法一：把4位的数字拆成四个数字 每个数字有10种可能性 所以标签就变成了长度为40的向量
# 方法二：多任务学习的方法 拆分成4个标签 每个标签长度是10

In [1]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image
from nets import nets_factory

In [4]:
#不同字符数量
CHAR_SET_LEN = 10
#图片高度、宽度
IMAGE_HEIGHT = 60
IMAGE_WIDTH = 160
#批次
BATCH_SIZE = 1
#tfrecord文件存放路径
TFRECORD_SAVE_PATH = 'G:/Anaconda3/My_Jupyter/captcha/train.tfrecords'
 
#占位符
x = tf.placeholder(tf.float32, [None, 224, 224])
y0 = tf.placeholder(tf.float32, [None])
y1 = tf.placeholder(tf.float32, [None])
y2 = tf.placeholder(tf.float32, [None])
y3 = tf.placeholder(tf.float32, [None])

# 学习率
lr = tf.Variable(0.003,dtype = tf.float32)   


# 之前我们生成了很多验证码图片
# 然后将它们变成了tfrecord类型的数据
# 从tfrecord读出数据
def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    #返回文件名和文件
    _,serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'image': tf.FixedLenFeature([], tf.string),
                                           'label0': tf.FixedLenFeature([], tf.int64),
                                           'label1': tf.FixedLenFeature([], tf.int64),
                                           'label2': tf.FixedLenFeature([], tf.int64),
                                           'label3': tf.FixedLenFeature([], tf.int64),
                                           })
    #获取图片数据
    image = tf.decode_raw(features['image'], tf.uint8)
    #tf.train.shuffle_batch必须确定shape
    image = tf.reshape(image, [224, 224])
    #图片预处理
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    #获取label
    label0 = tf.cast(features['label0'], tf.int32)
    label1 = tf.cast(features['label0'], tf.int32)
    label2 = tf.cast(features['label0'], tf.int32)
    label3 = tf.cast(features['label0'], tf.int32)
 
    return image, label0, label1, label2, label3



#获取图片数据和标签
image, label0, label1, label2, label3 = read_and_decode(TFRECORD_SAVE_PATH)
 
#使用shuffle_batch可以随机打乱
image_batch, label_batch0, label_batch1, label_batch2, label_batch3 = tf.train.shuffle_batch(
    [image, label0, label1, label2, label3], batch_size = BATCH_SIZE,
    capacity = 50000, min_after_dequeue=10000, num_threads=1) # 一个线程num_thread=1
 
#定义网络结构
train_network_fn = nets_factory.get_network_fn(
    name = 'alexnet_v2', 
    num_classes=CHAR_SET_LEN,
    weight_decay=0.0005,
    is_training=True)
 


In [5]:
with tf.Session() as sess:

    # 初始化    
    sess.run(tf.global_variables_initializer())

    # inputs: a tensor of size [batch_size, height, width, channels]
    X = tf.reshape(x, [BATCH_SIZE, 224, 224, 1])
    # 数据输入网络得到输出值
    logits0, logits1, logits2, logits3, end_points = train_network_fn(X)

    # 把标签转成one_hot的形式
    one_hot_labels0 = tf.one_hot(indices=tf.cast(y0, tf.int32), depth=CHAR_SET_LEN)
    one_hot_labels1 = tf.one_hot(indices=tf.cast(y1, tf.int32), depth=CHAR_SET_LEN)
    one_hot_labels2 = tf.one_hot(indices=tf.cast(y2, tf.int32), depth=CHAR_SET_LEN)
    one_hot_labels3 = tf.one_hot(indices=tf.cast(y3, tf.int32), depth=CHAR_SET_LEN)

    # 计算loss
    loss0 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits0, labels=one_hot_labels0))
    loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits1, labels=one_hot_labels1))
    loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits2, labels=one_hot_labels2))
    loss3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits3, labels=one_hot_labels3))
    # 计算总的loss
    total_loss = (loss0 + loss1 + loss2 + loss3) / 4.0
    # 优化total_loss
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)

    # 计算准确率
    correct_prediction0 = tf.equal(tf.argmax(one_hot_labels0, 1), tf.argmax(logits0, 1))
    accuracy0 = tf.reduce_mean(tf.cast(correct_prediction0, tf.float32))

    correct_prediction1 = tf.equal(tf.argmax(one_hot_labels1, 1), tf.argmax(logits1, 1))
    accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))

    correct_prediction2 = tf.equal(tf.argmax(one_hot_labels2, 1), tf.argmax(logits2, 1))
    accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))

    correct_prediction3 = tf.equal(tf.argmax(one_hot_labels3, 1), tf.argmax(logits3, 1))
    accuracy3 = tf.reduce_mean(tf.cast(correct_prediction3, tf.float32))

    
    
    # 用于保存模型
    saver = tf.train.Saver()


    # 创建一个协调器，管理线程
    coord = tf.train.Coordinator()
    # 启动QueueRunner, 此时文件名队列已经进队
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for i in range(4001):
        # 获取一个批次的数据和标签
        b_image, b_label0, b_label1, b_label2, b_label3 = sess.run([image_batch, label_batch0, label_batch1, label_batch2, label_batch3])
        # 优化模型
        sess.run(optimizer,feed_dict={x:b_image,y0:b_label0,y1:b_label1,y2:b_label2,y3:b_label3})

        # 每迭代20次计算一次loss和准确率
        if i % 20 == 0:
            # 每迭代2000次降低一次学习率
            if i % 2000 == 0:
                sess.run(tf.assign(lr, lr / 3))
            acc0, acc1, acc2, acc3, loss_ = sess.run([accuracy0, accuracy1, accuracy2, accuracy3, total_loss],
                                                     feed_dict={x: b_image,
                                                                y0: b_label0,
                                                                y1: b_label1,
                                                                y2: b_label2,
                                                                y3: b_label3})
            learning_rate = sess.run(lr)
            print('Iter:%d  Loss:%.3f  Accuracy:%.2f,%.2f,%.2f,%.2f  Learning_rate:%.4f' % (
            i, loss_, acc0, acc1, acc2, acc3, learning_rate))

            # 保存模型
            # 什么时候停止训练呢
            # 方法一：当所有准确率到达90%以上时
            # 方法二：当loos值比较小时 比如<0.001
            # 方法三：训练指定次数后停止
            # if acc0 > 0.90 and acc1 > 0.90 and acc2 > 0.90 and acc3 > 0.90:
            if i == 2000:
                saver.save(sess, 'G:/Anaconda3/My_Jupyter/captcha/models/crack_captcha.model', global_step=i)# 这个i会加到模型名称后
                break

                # 通知其他线程关闭
    coord.request_stop()
    # 其他所有线程关闭之后，这一函数才能返回
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/Identity/_9)]]


FailedPreconditionError: Attempting to use uninitialized value alexnet_v2/fc8_3/weights
	 [[Node: alexnet_v2/fc8_3/weights/read = Identity[T=DT_FLOAT, _class=["loc:@Adam/update_alexnet_v2/fc8_3/weights/ApplyAdam"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](alexnet_v2/fc8_3/weights)]]

Caused by op 'alexnet_v2/fc8_3/weights/read', defined at:
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\